In [43]:
import numpy as numpy
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [44]:
df = pd.read_csv('Churn_Modelling.csv')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [45]:
# Dropping the first 3 columns as they are not important
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [46]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [47]:
# Encoding the categorical features: Geography, Gender
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [48]:
# Applying OneHotEncoding to Geography column
one_hot_encoder_geography = OneHotEncoder(sparse_output=False)
geo_encoder = one_hot_encoder_geography.fit_transform(df[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [49]:
df = pd.concat([df, pd.DataFrame(data=geo_encoder, columns=one_hot_encoder_geography.get_feature_names_out())], axis=1)
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,France,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [50]:
# Dropping 'Geography' column
df.drop(columns=['Geography'], axis=1, inplace=True)
df.head(2)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int32  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int32(1), int64(7)
memory usage: 976.7 KB


In [52]:
# Saving the preprocessor files in the pkl format
def save_object(file_path, preprocessor_object):
    with open(file_path, 'wb') as file:
        pickle.dump(preprocessor_object, file)

In [53]:
# Saving LabelEncoder
save_object('label_encoder_gender.pkl', label_encoder_gender)

In [54]:
# Saving OneHotEncoder
save_object('onehot_encoder_geography.pkl', one_hot_encoder_geography)

In [55]:
# Independent and dependent features
X = df.drop(columns=['EstimatedSalary'], axis=1)
y = df['EstimatedSalary']

In [56]:
# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [57]:
# Standard Scaling on numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [58]:
# Saving the StandardScaler
save_object('standard_scaler.pkl', scaler)

### ANN Regression Model

In [59]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

In [69]:
# Building the Sequential model (ANN)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)    # Regression problem statement and by default activation='linear'
])

In [70]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.MeanAbsoluteError()

In [71]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=['mae']
)

In [72]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 64)                832       
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
# Setting up tensorboard for logging
log_dir = 'logs/fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [74]:
# Setting up earlystopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [75]:
# Model fitting and validation
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, 
                    callbacks=[tensorboard_callback, early_stopping_callback])

Epoch 1/100


250/250 [==============================] - 15s 12ms/step - loss: 56552.9883 - mae: 56552.9883 - val_loss: 51204.5000 - val_mae: 51204.5000
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 52441.5664 - mae: 52441.5664 - val_loss: 51509.6406 - val_mae: 51509.6406
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 52425.9453 - mae: 52425.9453 - val_loss: 51172.0156 - val_mae: 51172.0156
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 52522.0352 - mae: 52522.0352 - val_loss: 51088.8086 - val_mae: 51088.8086
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 52171.7852 - mae: 52171.7852 - val_loss: 51235.0234 - val_mae: 51235.0234
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 52178.0000 - mae: 52178.0000 - val_loss: 50989.5781 - val_mae: 50989.5781
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 52098.5039 - mae: 52098.503

In [81]:
# Load Tensorboard extension
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 21264), started 22:36:58 ago. (Use '!kill 21264' to kill it.)

In [82]:
# Evaluate the model using test data
test_loss, test_mae = model.evaluate(X_test, y_test)
test_loss, test_mae

63/63 [==============================] - 0s 4ms/step - loss: 50387.9375 - mae: 50387.9375


(50387.9375, 50387.9375)

In [84]:
# Saving the model
model.save('ann_model_reg.h5')  # h5 file which is compatible with Keras